In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Top 50 명령어 위주 추출 전처리

In [0]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train.pkl')
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test.pkl')

In [3]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0.0,1.0,0.0,0.0,0.0,0.0,"at (145.25, 21.5078125)"
1,0.0,1.0,0.0,1.0,0.0,0.0,"at (22.75, 147.0078125)"
2,0.0,1.0,0.0,0.0,0.0,1.0,['OrbitalCommand [3080001]']
3,0.0,1.0,0.0,0.0,0.0,2.0,(1360) - TrainSCV
4,0.0,1.0,0.0,0.0,0.0,0.0,"at (142.99609375, 24.50390625)"


In [4]:
test.head()

,game_id,time,player,species,event,event_contents
0,38872.0,0,0.0,1.0,0.0,"at (22.25, 81.5078125)"
1,38872.0,0,1.0,1.0,0.0,"at (120.25, 153.83984375)"
2,38872.0,0,1.0,1.0,1.0,['Nexus [3100001]']
3,38872.0,0,1.0,1.0,2.0,(15E0) - TrainProbe
4,38872.0,0,1.0,1.0,6.0,NaN


In [0]:
def top_ability(df,num):
  df['ability_content']=df[df['event']==2]['event_contents'].apply(lambda x: x[8:]).apply(lambda x: x.split(';')[0])
  data=df[df['event']==2][['game_id','winner','player','species','ability_content']]
  data=data.groupby('species')['ability_content'].value_counts().reset_index(name='count')
  
  return data[data['species']==0].head(num),data[data['species']==1].head(num),data[data['species']==2].head(num)

def make_df(ability_t,ability_z,ability_p):
  col_list=[]
  
  #종족별 상위 50개의 명령어를 col_list에 삽입
  for value in ability_t['ability_content']:
    col_list.append(value)
  for value in ability_z['ability_content']:
    col_list.append(value)
  for value in ability_p['ability_content']:
    col_list.append(value)
  
  # 중복제거를 위한 set처리 ex) attack
  col_list= list(set(col_list))

  # player 1과 2의 컬럼을 구분하기위해 col_list 변형 후,
  # new_colist에 컬럼이 될 것들을 넣고 컬럼 지정
  new_col_list=[]
  for col in col_list:
    for i in [0,1]:
      new_col_list.append('%s_%s'%(col,i))
  
  new_df = pd.DataFrame(columns=new_col_list)

  return new_df

def final_df(new_df,data):
  new_list =[]
  new_df_columns=new_df.columns
  id_list = list(set(data['game_id'].values))


  for id in id_list:
    # 연산 시간 측정
    if id % 1000 ==0:
      print(id)     
    
    '''game_id 별로 ability_content만 추출하자.'''
    data_0_values= list(data[(data['game_id']==id)&(data['player']==0)&(data['event']==2)]['ability_content'].values)
    #data_0_values= list(data.query("(game_id=={})&(player=={})&(event=={})".format(id,0,2))['ability_content'].values)  
    data_1_values= list(data[(data['game_id']==id)&(data['player']==1)&(data['event']==2)]['ability_content'].values)
    #data_1_values= list(data.query("(game_id=={})&(player=={})&(event=={})".format(id,1,2))['ability_content'].values)



    # 값을 넣기 위한 0 삽입
    new_dict = dict(zip(list(new_df.columns),[0 for _ in range(len(list(new_df_columns)))]))
    new_dict['game_id']=id
    
    # player 1 의 명령문 삽입
    for value in data_0_values:
      count = data_0_values.count(value)
      value= '%s_0'%(value)
      if value in new_df_columns:
        new_dict[value] = count
        #new_df.loc[id,value]+=1

    
    # player 2 의 명령문 삽입
    for value in data_1_values:
      count = data_1_values.count(value)
      value= '%s_1'%(value)
      if value in new_df_columns:
        new_dict[value] = count
        #new_df.loc[id,value]+=1
    
    new_list.append(new_dict)
        
  return new_list

In [0]:
%%time
a,b,c=top_ability(train,50)
new_df=make_df(a,b,c)
train_list=final_df(new_df,train)
pd.DataFrame(train_list).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train_top50_ability.csv')

a,b,c=top_ability(test,50)
new_df=make_df(a,b,c)
test_list=final_df(new_df,test)
pd.DataFrame(test_list).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top50_ability.csv')

0.0
